In [ ]:
from gcp_utils import constants
from firebase_admin import firestore, initialize_app
from gcp_utils.tools.utils import query_collection

initialize_app()

UID = 'wx1jF08b3DTPijtQcwGiEwpEFai2'

In [ ]:
# get all frames and windows in a user collection

database = firestore.client()
frames_ref = database.collection(f'bpm_data/{UID}/frames')
windows_ref = database.collection(f'bpm_data/{UID}/windows')

docs = query_collection(frames_ref, 'status', '==', 'processed')
frames = [d.to_dict() for d in docs]

docs = query_collection(windows_ref, 'status', '==', 'predicted')
windows = [d.to_dict() for d in docs]
docs = query_collection(windows_ref, 'status', '==', 'invalid')
for d in docs:
    windows.append(d.to_dict())

In [ ]:
import pickle as pkl

with open(None, 'wb') as f:
    pkl.dump(dict(windows=windows, frames=frames), f)

In [ ]:
database = firestore.client()
frames_ref = database.collection(f'bpm_data/{UID}/frames')

# Add document with no status
frame = constants.NEW_BPM_FRAME
frame.update({'status': ''})
frames_ref.add(frame)

 # Update status to trigger
doc = query_collection(frames_ref, 'fid', '==', '0')[0]
doc_ref = database.collection(f'bpm_data/{UID}/frames').document(doc.id)
doc_ref.update({'status': 'new'}) 

In [ ]:
import numpy as np
from database_tools.processing.detect import detect_peaks

database = firestore.client()
windows_ref = database.collection(f'bpm_data/{UID}/windows')

docs = query_collection(windows_ref, 'status', '==', 'predicted')
for d in docs:
    data = d.to_dict()
    abp = data['abp']
    peaks, troughs = detect_peaks(abp).values()
    print(f'sbp = {np.mean(np.array(abp)[peaks])}, sbp = {np.mean(np.array(abp)[troughs])}')

In [ ]:
import pickle as pkl

with open('bpm_scaler.pkl', 'wb') as f:
    pkl.dump((0, 5600), f)